In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [2]:
import torch
import torch.nn as nn

In [3]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [4]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = 
                                                                                       GLOVE_EMBEDDING)

In [5]:
batch_size = 64
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=False, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=False, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=False, repeat=False)

In [6]:
val_list = list(val_iter)

In [7]:
batch = val_list[50]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

<sos> predictable and boring <eos> | other
<sos> very interesting . <eos> | other
<sos> not scary . <eos> | other
<sos> awful <unk> disturbing <eos> | other
<sos> same story . <eos> | plot
<sos> see this . <eos> | other
<sos> good movie . <eos> | other
<sos> worth the watch <eos> | other
<sos> what a ride <eos> | other
<sos> ¥ â ? <eos> | other


In [8]:
#review_text_FIELD.vocab.vectors.shape, len(review_text_FIELD.vocab.itos)

In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 1024
layers = 2
dropout = .5

vocab_size, label_size, emb_dim, vectors.shape

(20004, 5, 300, torch.Size([20004, 300]))

In [11]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim + emb_lab, nhid, nlayers, dropout=dropout)
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)

        if pretrained:
            self.word_embedding.weight.data.copy_(vectors)
            self.word_embedding.from_pretrained(GLOVE_EMBEDDING.vectors)
            
        self.init_weights()    
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        #print('R.shape', reviews.shape)
        R = self.word_embedding(reviews)
        L = self.label_embedding(labels)
        L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        X = torch.cat([R, L], -1)
        #print('X.shape', X.shape)
        
        X = self.drop(X)
        X, hidden = self.rnn(X, hidden)
        X = self.drop(X)
        #print('X.shape', X.shape)
        #X = X.view(X.size(0)*X.size(1), X.size(2))
        #print('X.shape', X.shape)
        
        X = self.decoder(X)
        #print('X.shape', X.shape)
        log_probs = torch.softmax(X, -1)
        return log_probs, hidden
      

In [12]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [13]:
import numpy as np
import gc

In [14]:
def evaluate(model, data_source, criterion):
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0

    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[1:-1,:], batch[2:,:]
                
                output, hidden = model(data, labels, hidden)
                output_flat = output.contiguous().view(-1, vocab_size)
                
                batch_loss = criterion(output_flat, targets.contiguous().view(-1)).detach().item()

                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss
                total_number_of_words += number_of_words

                hidden = repackage_hidden(hidden)
            
    return total_loss_e / total_number_of_words

In [15]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf')):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[1:-1,:], batch[2:,:]
                
                optimizer.zero_grad()
                output, hidden = model(data, labels, hidden)
                hidden = repackage_hidden(hidden)

                output_flat = output.contiguous().view(-1, vocab_size)
                batch_loss = criterion(output_flat, targets.contiguous().view(-1))
                number_of_words = data.shape[0] * data.shape[1]

                total_loss_e += batch_loss.detach().item()
                total_number_of_words += number_of_words
            
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                batch_loss.backward()
                optimizer.step()

                if i % 500 == 0:
                    cur_loss = batch_loss.item() / number_of_words #np.mean(total_loss_e)#
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                #gc.collect()
                #torch.cuda.empty_cache()
                
                if i % 4999 == 1: #len_train_iter - 1:
                    cur_loss = total_loss_e / total_number_of_words #np.mean(total_loss_e) #
                    tr_ppl_print = np.exp(cur_loss)
                    gc.collect()
                    #torch.cuda.empty_cache()
                    val_loss_eval = evaluate(model, dev_iter, criterion)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = 'best_model_{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()
                    
                if i == 10000: break


In [16]:
#with open('./best_model_base_model_ppl_15624.350597914712.model', 'rb') as file:
#    model = torch.load(file)#BaseModel().cuda()
#    model.eval()

In [17]:
#with open('best_model_name', 'rb') as file:
#    model = torch.load(file) 

In [18]:
model = BaseModel(pretrained=True).cuda()

In [19]:
model

BaseModel(
  (drop): Dropout(p=0.5)
  (word_embedding): Embedding(20004, 300)
  (label_embedding): Embedding(5, 20)
  (rnn): LSTM(320, 1024, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=20004, bias=True)
)

In [20]:
# implement PPL
learning_rate = 0.01
criterion = nn.CrossEntropyLoss(reduction='sum',
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

In [21]:
#np.exp(evaluate(model, val_list, criterion))

In [ ]:
train(model,
      ep0 = 1,
      epN = 3,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'base_model_ppl',
      best_ppl = float('inf'))

| epoch   0 | batch 500 / 10981 | train_loss 9.679 | train_ppl 15979.841
| epoch   0 | batch 1000 / 10981 | train_loss 9.716 | train_ppl 16585.358
| epoch   0 | batch 1500 / 10981 | train_loss 9.762 | train_ppl 17353.177
| epoch   0 | batch 2000 / 10981 | train_loss 9.759 | train_ppl 17316.581
| epoch   0 | batch 2500 / 10981 | train_loss 9.676 | train_ppl 15925.449
| epoch   0 | batch 2500 / 10981 | train_loss 9.706 | train_ppl 16414.26 | val_loss 9.757 | val_ppl 17281.964
old best ppl 26821.121 new best ppl 17281.963849636402
save model... best_model_base_model_ppl_17281.963849636402.model


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   0 | batch 3000 / 10981 | train_loss 9.856 | train_ppl 19077.485
| epoch   0 | batch 3500 / 10981 | train_loss 9.802 | train_ppl 18062.801
| epoch   0 | batch 4000 / 10981 | train_loss 9.748 | train_ppl 17117.231
| epoch   0 | batch 4500 / 10981 | train_loss 9.769 | train_ppl 17486.194
| epoch   0 | batch 5000 / 10981 | train_loss 9.771 | train_ppl 17512.124
| epoch   0 | batch 5500 / 10981 | train_loss 9.777 | train_ppl 17618.156
| epoch   0 | batch 6000 / 10981 | train_loss 9.776 | train_ppl 17601.243
| epoch   0 | batch 6500 / 10981 | train_loss 9.725 | train_ppl 16723.1
| epoch   0 | batch 7000 / 10981 | train_loss 9.739 | train_ppl 16962.161
| epoch   0 | batch 7500 / 10981 | train_loss 9.738 | train_ppl 16946.575
| epoch   0 | batch 8000 / 10981 | train_loss 9.742 | train_ppl 17013.963
| epoch   0 | batch 8500 / 10981 | train_loss 9.75 | train_ppl 17156.829
| epoch   0 | batch 9000 / 10981 | train_loss 9.77 | train_ppl 17501.805
| epoch   0 | batch 9500 / 10981 | train_l

KeyboardInterrupt: 

| epoch  14 | batch 500 / 10981 | train_loss 9.716 | train_ppl 16573.915
| epoch  14 | batch 1000 / 10981 | train_loss 9.718 | train_ppl 16615.205
| epoch  14 | batch 1500 / 10981 | train_loss 9.714 | train_ppl 16546.362
| epoch  14 | batch 2000 / 10981 | train_loss 9.706 | train_ppl 16423.484
| epoch  14 | batch 2500 / 10981 | train_loss 9.699 | train_ppl 16305.76
| epoch  14 | batch 3000 / 10981 | train_loss 9.709 | train_ppl 16473.007
| epoch  14 | batch 3500 / 10981 | train_loss 9.719 | train_ppl 16627.063
| epoch  14 | batch 4000 / 10981 | train_loss 9.696 | train_ppl 16248.309
| epoch  14 | batch 4500 / 10981 | train_loss 9.716 | train_ppl 16576.229
| epoch  14 | batch 5000 / 10981 | train_loss 9.734 | train_ppl 16889.702
| epoch  14 | batch 5000 / 10981 | train_loss 9.714 | train_ppl 16541.947 | val_loss 9.657 | val_ppl 15633.254
| epoch  14 | batch 5500 / 10981 | train_loss 9.736 | train_ppl 16917.136
| epoch  14 | batch 6000 / 10981 | train_loss 9.748 | train_ppl 17119.425
| e

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch  14 | batch 10000 / 10981 | train_loss 9.739 | train_ppl 16963.728
| epoch  15 | batch 500 / 10981 | train_loss 9.715 | train_ppl 16562.513
| epoch  15 | batch 1000 / 10981 | train_loss 9.71 | train_ppl 16482.687
| epoch  15 | batch 1500 / 10981 | train_loss 9.714 | train_ppl 16544.264
